## 1 - Transforme a seguinte GIC em notação lark :
```
 T  = { '[', ']', STRING, DATA, FICH }
         P = {
         p1: album  : capa "[" pags "]" ccapa
         p2: capa   : titulo autor DATA
         p3: titulo : STRING
         p4: autor  : STRING
         p5: ccapa  : fecho DATA
         p6: fecho  : STRING
         p7: pags   : pag+
         p8: pag    : sep | folha
         p9: sep    : titulo
         p10: folha : foto+
         p11: foto  : FICH legenda
         p12: legenda : STRING
         }
```
Considere a seguinte frase de exemplo válida :
```
        "Um passeio pelo Geres" 
        Joaozinho 
        03-04-2000 
        [ 
          "Cascata do tahiti"
          img1
          "Vista da cascata."
           
          "A aldeia de Soajo"
          img2 
          "casa da aldeia."
        ] 
        "Dedicado à família" 
        03-05-2000
```

In [122]:
from lark import Discard
from lark import Lark,Token,Tree
from lark.tree import pydot__tree_to_png
from lark.visitors import Interpreter
import datetime

grammar = '''
album  : capa PE pags PD ccapa
capa   : titulo autor DATA
titulo : STRING_1
autor  : STRING_2
pags   : pag+
pag    : sep
       | folha
sep    : titulo
folha : foto+
foto  : FICH legenda
legenda : STRING_1
ccapa  : fecho DATA
fecho  : STRING_1
PE: "["
PD: "]"
STRING_1: ESCAPED_STRING
STRING_2: /.+/
DATA: INT "-" INT "-" INT
FICH: /[a-zA-Z0-9_]+/
%import common.WS
%import common.ESCAPED_STRING
%import common._STRING_ESC_INNER
%import common.WORD
%import common.INT
%ignore WS
'''

In [123]:
class MyInterpreter(Interpreter):
  count = 0
  dic_turmas = {}
  num_disciplinas = 0
    
  def album(self,elem):
    info = []
    for item in elem.children:
      if isinstance(item,Tree):
        info.append(self.visit(item))
        
    return info
 
  def capa(self,elem):
    capa = {}
    for item in elem.children:
      if isinstance(item,Tree):
        # titulo e autor
        capa[str(item.data)] = str(item.children[0]).strip()
      else:
        capa[str(item.type)] = str(item).strip()
    return ("capa",capa)
 
  def pags(self,elem):
    pags = []
    for item in elem.children:
      if isinstance(item,Tree):
        pags.append(self.visit(item))

    return ("pags",pags)
 
  def pag(self,elem):
    return self.visit(elem.children[0])
 
  def sep(self,elem):
    titulo = elem.children[0].children[0]
    return ("sep",str(titulo).strip())
 
  def folha(self,elem):
    folha = []
    for item in elem.children:
      if isinstance(item,Tree):
        folha.append(self.visit(item))
      else:
        folha.append(("descri",str(item).strip()))

    return ("folha",folha)

 
  def foto(self,elem):
    return (str(elem.children[0]).strip(),self.visit(elem.children[1]))
   
  def legenda(self,elem):
    return str(elem.children[0]).strip()
  
  def ccapa(self,elem):
    ccapa = {}
    for item in elem.children:
      if isinstance(item,Tree):
        # titulo e autor
        ccapa[str(item.data)] = str(item.children[0]).strip()
      else:
        ccapa[str(item.type)] = str(item).strip()
    return ("ccapa",ccapa)

In [124]:
frase = """
"Um passeio pelo Geres" 
Joaozinho 
03-04-2000 
[ 
    "Cascata do tahiti"
    img1
    "Vista da cascata."
    
    "A aldeia de Soajo"
    img2 
    "casa da aldeia."
] 
"Dedicado à família" 
03-05-2000
"""


# Inicializar o objeto lark
p = Lark(grammar,start="album")
# Criar a parsing tree
parse_tree = p.parse(frase)
# Utilizar o Interpreter para visitar a parsing tree
data=MyInterpreter().visit(parse_tree)

print(data)

[('capa', {'titulo': '"Um passeio pelo Geres"', 'autor': 'Joaozinho', 'DATA': '03-04-2000'}), ('pags', [('sep', '"Cascata do tahiti"'), ('folha', [('img1', '"Vista da cascata."')]), ('sep', '"A aldeia de Soajo"'), ('folha', [('img2', '"casa da aldeia."')])]), ('ccapa', {'fecho': '"Dedicado à família"', 'DATA': '03-05-2000'})]



## 2 - Imprimir o album para latex, considerando :

  -  que o titulo deverá ser um \title ;
  -  que o autor deverá ser um \author ;
  -  cada página deverá ser um \title e terminar com o \newpage ;
  - uma foto devera usar o nome do ficheiro para renderizar e a legenda como \caption.Exemplo :
  ```
\begin{figure}[h!]
\centering
\includegraphics[width=0.3\textwidth]{frog.jpg}
\caption{\label{fig:frog}Vista da cascata.}
\end{figure}
```

In [127]:
def trataCapa(capa):
    latex = ""
    for key,val in capa.items():
        if key == "titulo":
            latex += "\\title{" + val + "}"

        if key == "autor":
            latex += "\\author{" + val + "}"

        if key == "DATA":
            latex += "\\date{" + val + "}"

    return latex

def trataPags(pags):
    latex = ""
    for val in pags:
        if val[0] == "sep":
            latex += "\\title{" + val[1] + "}"

        elif val[0] == "folha":
            for foto in val[1]:
                if foto[0] != "descri":
                    latex += "\\begin{figure}[h!]"
                    latex += "\\centering"
                    latex += "\\includegraphics[width=0.3\\textwidth]{" + foto[0] + "}"
                    latex += "\\caption{\label{fig:" + foto[0] + "}" + foto[1] + "}"
                    latex += "\\end{figure}"
                else:
                    latex += foto[1]
        
    latex += "\\newpage"

    return latex
    

def trataCCapa(ccapa):
    latex = ""
    for key,val in ccapa.items():
        if key == "fecho":
            latex += "\\title{" + val + "}"

        if key == "DATA":
            latex += "\\date{" + val + "}"
    return latex

res = ""
for elem in data:
    if elem[0] == "capa":
        res += trataCapa(elem[1])
    elif elem[0] == "pags":
        res += trataPags(elem[1])
    elif elem[0] == "ccapa":
        res += trataCCapa(elem[1])

print(res)

\title{"Um passeio pelo Geres"}\author{Joaozinho}\date{03-04-2000}\title{"Cascata do tahiti"}\begin{figure}[h!]\centering\includegraphics[width=0.3\textwidth]{img1}\caption{\label{fig:img1}"Vista da cascata."}\end{figure}\title{"A aldeia de Soajo"}\begin{figure}[h!]\centering\includegraphics[width=0.3\textwidth]{img2}\caption{\label{fig:img2}"casa da aldeia."}\end{figure}\newpage\title{"Dedicado à família"}\date{03-05-2000}


## 3 - Extender a linguagem para permitir que uma folha possa também ter texto descritivo do momento retratado na foto.

In [126]:
from lark import Discard
from lark import Lark,Token,Tree
from lark.tree import pydot__tree_to_png
from lark.visitors import Interpreter
import datetime

grammar = '''
album  : capa PE pags PD ccapa
capa   : titulo autor DATA
titulo : STRING_1
autor  : STRING_2
pags   : pag+
pag    : sep
       | folha
sep    : titulo
folha : foto+ DESCRI?
foto  : FICH legenda
legenda : STRING_1
ccapa  : fecho DATA
fecho  : STRING_1
PE: "["
PD: "]"
DESCRI: "-" STRING_1
STRING_1: ESCAPED_STRING
STRING_2: /.+/
DATA: INT "-" INT "-" INT
FICH: /[a-zA-Z0-9_]+/
%import common.WS
%import common.ESCAPED_STRING
%import common._STRING_ESC_INNER
%import common.WORD
%import common.INT
%ignore WS
'''


## 4 - Atualizar a impressão do livro para latex.

In [ ]:
def trataCapa(capa):
    latex = ""
    for key,val in capa.items():
        if key == "titulo":
            latex += "\\title{" + val + "}"

        if key == "autor":
            latex += "\\author{" + val + "}"

        if key == "DATA":
            latex += "\\date{" + val + "}"

    return latex

def trataPags(pags):
    latex = ""
    for val in pags:
        if val[0] == "sep":
            latex += "\\title{" + val[1] + "}"

        elif val[0] == "folha":
            for foto in val[1]:
                if foto[0] != "descri":
                    latex += "\\begin{figure}[h!]"
                    latex += "\\centering"
                    latex += "\\includegraphics[width=0.3\\textwidth]{" + foto[0] + "}"
                    latex += "\\caption{\label{fig:" + foto[0] + "}" + foto[1] + "}"
                    latex += "\\end{figure}"
                else:
                    latex += foto[1]
                
    latex += "\\newpage"

    return latex
    

def trataCCapa(ccapa):
    latex = ""
    for key,val in ccapa.items():
        if key == "fecho":
            latex += "\\title{" + val + "}"

        if key == "DATA":
            latex += "\\date{" + val + "}"
    return latex

res = ""
for elem in data:
    if elem[0] == "capa":
        res += trataCapa(elem[1])
    elif elem[0] == "pags":
        res += trataPags(elem[1])
    elif elem[0] == "ccapa":
        res += trataCCapa(elem[1])

print(res)